In [8]:
using Revise
using PowerModels
using Ipopt
using ForwardDiff
#using OrdinaryDiffEq
using NLsolve
using PowerSystems
const PSY = PowerSystems

PowerSystems

In [9]:
display(HTML("<style>.container { width:100% !important; }</style>"))

HTML{String}("<style>.container { width:100% !important; }</style>")

In [21]:
file_dir = joinpath(pwd(), "Twogen2bus.m")
sys = System(PSY.PowerModelsData(file_dir))
solve_powerflow!(sys, nlsolve, method = :newton)

┌ Info: Constructing System from Power Models
│   data["name"] = Twogen2bus
│   data["source_type"] = matpower
└ @ PowerSystems /Users/jdlara/.julia/packages/PowerSystems/wHSk9/src/parsers/power_models_data.jl:33
┌ Info: Reading bus data
└ @ PowerSystems /Users/jdlara/.julia/packages/PowerSystems/wHSk9/src/parsers/power_models_data.jl:122
┌ Info: Reading generator data
└ @ PowerSystems /Users/jdlara/.julia/packages/PowerSystems/wHSk9/src/parsers/power_models_data.jl:368
┌ Warning: Generator cost data not included for Generator: gen-1-1
└ @ PowerSystems /Users/jdlara/.julia/packages/PowerSystems/wHSk9/src/parsers/power_models_data.jl:330
┌ Info: Reading branch data
└ @ PowerSystems /Users/jdlara/.julia/packages/PowerSystems/wHSk9/src/parsers/power_models_data.jl:520
┌ Warning: Rate provided for 1 is 32.0, larger the SIL 258.94053053874836 in the range of (min = 325.0, max = 425.0).
└ @ PowerSystems /Users/jdlara/.julia/packages/PowerSystems/wHSk9/src/utils/IO/branchdata_checks.jl:150
┌ 

true

In [26]:
for b in get_components(Bus, sys)
    println("Vm = $(get_voltage(b))", " ","Va= $(get_angle(b))")
end

Vm = 1.02 Va= 0.0
Vm = 1.0 Va= -0.1279338657728345


In [19]:
data = PowerModels.parse_file(file_dir)
opt = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)
pf_result = run_ac_pf(data, opt)
PowerModels.print_summary(pf_result["solution"])

Metadata
  baseMVA: 1000
  per_unit: true

Table Counts
  bus: 2
  gen: 1


Table: bus
            vm,     va
      1: 1.020,  0.000
      2: 1.000, -0.128


Table: gen
            pg,    qg
      1: 0.703, 0.051


In [ ]:
# Load flow results

# bus results
va1 =pf_result["solution"]["bus"]["1"]["va"]
va2 =pf_result["solution"]["bus"]["2"]["va"]
vm1 =pf_result["solution"]["bus"]["1"]["vm"]
vm2 =pf_result["solution"]["bus"]["2"]["vm"];

# Voltage at bus 1
V_tR = vm1*cos(va1)
V_tI = vm1*sin(va1)


# Voltage at bus 2
V_tR1 = vm2*cos(va2)
V_tI1 = vm2*sin(va2)

# generator results
pg = pf_result["solution"]["gen"]["1"]["pg"]
qg =pf_result["solution"]["gen"]["1"]["qg"]

# branch results
p_ref =pf_result["solution"]["branch"]["1"]["pf"]
q_ref =pf_result["solution"]["branch"]["1"]["qf"]
p_ref1 =pf_result["solution"]["branch"]["1"]["pt"]
q_ref1 =pf_result["solution"]["branch"]["1"]["qt"];

In [ ]:
# Define the parameters
lf =0.08
rf = 0.003
cf = 0.074
rt = 0.2
lt = 0.01
rl = 0.01

# Line parameters
# What values should be used?
ick_d =  0.01
ick_q = 0.01
icj_d = 0.02
icj_q = 0.02
c_lk = 0.02
l_lk = 0.1
r_lk = 0.01
c_lj = 0.02
l_lj = 0.1
r_lj = 0.01
r_jk = 0.01
l_jk = 0.02


kp_pll = 0.4
ki_pll = 4.69

ωb = 2*pi*50.0
ωb1 = 2*pi*50.0

Rp = 0.1
Rq =0.1
ωz = 2*pi*5.0
ωz1 = 2*pi*5.0
ωg = 1.0

kvp = 0.52
kvi = 1.16
kvf = 1.0
rv = 0.0
lv =0.2
kip = 0.74
kii = 1.19
kif = 1.0

ω_ref = 1.0
v_ref = 0.8
ωn = 2*pi*50.0
ωg =1.0
ω_star = ωg


    function system!(F,x)
###Grid following equations
    #𝜕ed/𝜕t
    F[1] = (ωb / cf * x[3] - ωb / cf * x[5] + ωb * ωg * x[2])
    #𝜕eq/𝜕t
    F[2] = (ωb / cf * x[4] - ωb / cf * x[6] - ωb* ωg *x[1])
    #𝜕isd/𝜕t
    F[3] = (ωb / lf * (  kip * ( (kvp * (((v_ref + Rq*(q_ref1 - x[11]))- rv * x[5] + ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10]))* lv * x[6]) - x[1]) + kvi * x[12]
    - cf *   ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10]))* x[2] + kif * x[5]) - x[3]) + kii * x[14] - (ω_star + Rp*(p_ref1 - x[10])) * lf * x[4] + kvf *x[1])  - ωb / lf *x[1] - ωb * rf / lf * x[3] + ωb * ωg * x[4])
    #𝜕isq/𝜕t
    F[4] = (ωb / lf * ((kip * ((kvp * ((-rv * x[6] -  ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10])) * lv * x[5]) - x[2]) + kvi * x[13] + cf * ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10])) *x[1] + kif * x[6]) - x[4])
    + kii * x[15] +  ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10]))* lf * x[3] + kvf * x[2] )) - ωb / lf * x[2] - ωb * rf / lf * x[4] - ωb * ωg * x[3])
    #𝜕igd/𝜕t
    F[5] = (ωb / lt *x[1] - ωb / lt * (rl*x[6]) - ωb * rt / lt * x[5] + ωb * ωg * x[6])
    #𝜕igq/𝜕t
    F[6] = (ωb / lt * x[2] + ωb / lt *(rl*x[5])  - ωb * rt / lt * x[6] - ωb * ωg * x[5])
    # PLL states
    #𝜕δc/𝜕t
    F[7] = (((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10]))- ωg)*ωb
    #𝜕δpll/𝜕t
    F[8] = (ωn - ωg + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9])*ωb
    #𝜕ϵ_pll/𝜕t
    F[9] = x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])
    # Outer control model - virtual inertia
    #𝜕pm/𝜕t
     F[10] = (ωz*x[6]*x[2] + ωz*x[5]*x[1]- ωz*x[10])
    #𝜕qm/𝜕t
    F[11] = (-ωz * x[6] *x[1] + ωz * x[5] * x[2] - ωz * x[11])
    # Voltage control
    #PI Integrator (internal state)
    #𝜕ξ_d/𝜕t
    F[12] = (((v_ref + Rq*(q_ref1 - x[11]))  - rv * x[5] + ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10]))* lv * x[6]) -x[1])
     #𝜕ξ_q/𝜕t
     F[13] = (-rv * x[6] -  ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10])) - x[2])
    # Current control
    # PI integrator (internal state)
    #𝜕γ_d/𝜕t
     F[14] = (kvp * (((v_ref + Rq*(q_ref1 - x[11])) - rv * x[5] +  ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10]))* lv * x[6]) -x[1]) + kvi * x[12] - cf *  ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10])) * x[2] + kif * x[5]) - x[3]
    #𝜕γ_q/𝜕t
    F[15] = (kvp * ((-rv * x[6] -  ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10])) * lv * x[5]) - x[2]) + kvi * x[13] + cf * ((ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]) + Rp*(p_ref1 - x[10])) *x[1] + kif * x[6]) - x[4]

    vnjq =(rl*x[31])*sin(x[7]-x[8]) +(rl*x[30])*cos(x[8]-x[7])
    vnjd =(rl*x[31])*cos(x[8]-x[7]) +(rl*x[30])*sin(x[8]-x[7])
    vnkd = rl*x[6]
    vnkq = rl*x[5]
    
###Line equations
    #grid following side
    #𝜕vnjd/𝜕t
     F[16] = (ωb /c_lj)*icj_d - (ω_star + Rp*(p_ref - x[33]))*ωb*vnjq
     #𝜕vnjq/𝜕t
     F[17] = (ωb /c_lj)*icj_q + (ω_star + Rp*(p_ref - x[33]))*ωb*vnjd
     #𝜕iljd/𝜕t 
     F[18] = (ωb /l_lj)*vnjd - (r_lj/(l_lj*ωb) - ωb*(ω_star + Rp*(p_ref - x[33])))*x[19]
     #𝜕iljq/𝜕t 
     F[19] = (ωb /l_lj)*vnjq - (r_lj/(l_lj*ωb) + ωb*(ω_star + Rp*(p_ref - x[33])))*x[18]
    #grid forming side
     #𝜕vnkd/𝜕t
     F[20] =(ωb /c_lk)*ick_d - (ω_star + Rp*(p_ref - x[33]))*ωb*vnkq
     #𝜕vnkq/𝜕t 
     F[21] =(ωb /c_lk)*ick_q + (ω_star + Rp*(p_ref - x[33]))*ωb*vnkd
     #𝜕ilkd/𝜕t 
     F[22] =(ωb /l_lk)*- (r_lk/(l_lk*ωb) - ωb*(ω_star + Rp*(p_ref - x[33])) )*x[23]
     #𝜕ilkq/𝜕t  
     F[23] =(ωb /l_lk)*- (r_lk/(l_lk*ωb) + ωb*(ω_star + Rp*(p_ref - x[33])) )*x[22]
    #line dynamic
     #𝜕ijkd/𝜕t 
     F[24] =(ωb /l_jk)*(vnjd - vnkd)- ωb*(r_jk/l_jk + ωb*(ω_star + Rp*(p_ref - x[33])))*x[25]
     #𝜕ijkq/𝜕t  
     F[25] =(ωb /l_jk)*(vnjq - vnkq) - ωb*(r_jk/l_jk - ωb*(ω_star + Rp*(p_ref - x[33])) )*x[24]
 
### Grid forming equations
   #𝜕eg/𝜕t
 F[26]= (ωb1 / cf * x[28] - ωb1 / cf * x[30] + ωb1 * ωg * x[27])
    #𝜕eq/𝜕t
 F[27]= (ωb1 / cf * x[29] - ωb1 / cf * x[31] - ωb1 * ωg *x[26])
    #𝜕isd/𝜕t
 F[28]=(ωb1 / lf * (  kip * ( (kvp * (((v_ref + Rq*(q_ref - x[34]))- rv * x[30] + (ω_star + Rp*(p_ref - x[33])) * lv * x[31]) -x[26]) + kvi * x[35] - cf *  (ω_star + Rp*(p_ref - x[33])) * x[27] + kif * x[30]) - x[28]) + kii * x[37] - (ω_star + Rp*(p_ref - x[33])) * lf * x[29] + kvf *x[26])  - ωb1 / lf *x[26] - ωb1 * rf / lf * x[28] + ωb1 * ωg * x[29])
    #𝜕isq/𝜕t
 F[29]=    (ωb / lf * ((kip * ((kvp * ((-rv * x[31] - (ω_star + Rp*(p_ref - x[33])) * lv * x[30]) - x[27]) + kvi * x[36] + cf *  (ω_star + Rp*(p_ref - x[33])) *x[26] + kif * x[31]) - x[29]) + kii * x[38] +  (ω_star + Rp*(p_ref - x[33]))* lf * x[28] + kvf * x[27] )) - ωb1 / lf * x[27] - ωb1 * rf / lf * x[29] - ωb1 * ωg * x[28])
    #𝜕igd/𝜕t
 F[30]=  (ωb1 / lt *x[26] - ωb1 / lt * (rl*x[31]) - ωb1 * rt / lt * x[30] + ωb1 * ωg * x[31])
    #𝜕igq/𝜕t
    F[31]= (ωb1 / lt * x[27] + ωb1 / lt * (rl*x[30]) - ωb1 * rt / lt * x[31] - ωb1 * ωg * x[30])
    # Outer control model - virtual inertia
    #𝜕pm/𝜕t
 F[32]=  (ωz1*x[31]*x[27] + ωz1*x[30]*x[26]- ωz1*x[33])
    #𝜕qm/𝜕t
 F[33]= (-ωz1 * x[31] *x[26] + ωz1 * x[30] * x[27] - ωz1 * x[34])
   #𝜕δc/𝜕t
    F[34]= ωb*(ω_star + Rp*(p_ref - x[33]))
    # Voltage control
    #PI Integrator (internal state)
    #𝜕ξ_d/𝜕t
F[35]=     (((v_ref + Rq*(q_ref - x[34]))  - rv * x[30] +  (ω_star + Rp*(p_ref - x[33])) * lv * x[31]) -x[26])
    #𝜕ξ_q/𝜕t
F[36]=     ((-rv * x[31] - (ω_star + Rp*(p_ref - x[33])) * lv * x[30]) - x[27])
    # Current control
    # PI integrator (internal state)
    #𝜕γ_d/𝜕t
F[37]=     (kvp * (((v_ref + Rq*(q_ref - x[34])) - rv * x[30] +  (ω_star + Rp*(p_ref - x[33])) * lv * x[31]) -x[26]) + kvi * x[35] - cf *  (ω_star + Rp*(p_ref - x[33])) * x[27] + kif * x[30]) - x[28]
     #𝜕γ_q/𝜕t
    F[38]= (kvp * ((-rv * x[31] -  (ω_star + Rp*(p_ref - x[33])) * lv * x[30]) - x[27]) + kvi * x[36] + cf * (ω_star + Rp*(p_ref - x[33])) *x[26] + kif * x[31]) - x[29]
end

# Initial Conditions
     initial =  [ V_tR, #egd
            V_tI, #egq
            0.0, #isd
            0.0, #isq
            p_ref1/V_tR, #igd
            0.0,#igq
            0.0, #δc
            0.0, #δpll
            0.0, #ϵ_pll
            p_ref1, #pm
            q_ref1, #qm
            0.0015, #ξ_d
            0.0, #ξ_q
            0.0, #γ_d
            0.0, #γ_d
            0.5, #vnjd
            0.0, #vnjq
           -0.10, #iljd
            0.49, #iljd
           -0.1,#vnkd
            0.0, #vnkq
           -0.10, #ilkd
            0.49, #ilkq
           -0.1, #ijkd
            0.10,#ijkq
            V_tR1, #egd1
           V_tI1, #egq1
            0.5, #isd1
            0.0, #isq1
            p_ref/V_tR1, #igd1
           0.0,#igq1
            p_ref, #pm1
            q_ref, #qm1
            0.0,#δc1
            0.0, #ξ_d1
            0.0, #ξ_q1
            0.0, #γ_d1
            0.0, #γ_q1
         ]


         sys_solve = NLsolve.nlsolve(
                     system!,
                     initial,
                     xtol = :1e-9,
                     ftol = :1e-9,
                     method = :trust_region,
                    ) #Solve using initial guess x0
                      print(sys_solve)